In [1]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="pwucV0qVeD2nO2PjRqon")
# project = rf.workspace("smart-farm-juaeh").project("iam-rdpzb")
# version = project.version(3)
# dataset = version.download("yolov8")

In [2]:
# # !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="DEv4eiG6UDnTdSLAgiI5")
# project = rf.workspace("iam-z99dk").project("iam_crop_with_padding-50")
# version = project.version(1)
# dataset = version.download("yolov8")
                

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

def calculate_corners_from_points(points):
    """
    탐지된 점들로부터 정확한 좌상, 좌하, 우상, 우하 꼭짓점을 계산.
    Args:
        points: (numpy array) 점들의 좌표 리스트 [(x1, y1), (x2, y2), ...].
    Returns:
        (tuple) 좌상, 좌하, 우상, 우하 꼭짓점 좌표.
    """
    points = np.array(points)

    # 1. 좌상(Top-Left): x + y가 최소인 점
    top_left = tuple(points[np.argmin(points[:, 0] + points[:, 1])])

    # 2. 좌하(Bottom-Left): x - y가 최소인 점
    bottom_left = tuple(points[np.argmin(points[:, 0] - points[:, 1])])

    # 3. 우상(Top-Right): x - y가 최대인 점
    top_right = tuple(points[np.argmax(points[:, 0] - points[:, 1])])

    # 4. 우하(Bottom-Right): x + y가 최대인 점
    bottom_right = tuple(points[np.argmax(points[:, 0] + points[:, 1])])

    return top_left, bottom_left, top_right, bottom_right
    

def crop_include_points_and_resize(image, points, target_size=(1280, 960)):
    # 1. 네 꼭짓점의 최소/최대 좌표 계산
    min_x = min(point[0] for point in points)
    max_x = max(point[0] for point in points)
    min_y = min(point[1] for point in points)
    max_y = max(point[1] for point in points)

    '''좌우 여백 추가'''
    
    padding_x = 50  # 좌우 각각 200픽셀 추가
    min_x = max(0, min_x - padding_x)
    max_x = min(image.shape[1], max_x + padding_x)

    # 2. 네 꼭짓점을 포함하는 영역의 중심 계산
    center_x = (min_x + max_x) // 2
    center_y = (min_y + max_y) // 2

    # 3. 영역의 너비와 높이를 기반으로 4:3 비율 계산
    original_height, original_width = image.shape[:2]
    crop_width = max_x - min_x
    crop_height = max_y - min_y

    # 4:3 비율 맞추기
    target_ratio = 4 / 3
    if crop_width / crop_height > target_ratio:
        crop_height = int(crop_width / target_ratio)
    else:
        crop_width = int(crop_height * target_ratio)

    # 중심을 기준으로 잘라낼 좌표 계산
    x_min = max(0, center_x - crop_width // 2)
    x_max = min(original_width, center_x + crop_width // 2)
    y_min = max(0, center_y - crop_height // 2)
    y_max = min(original_height, center_y + crop_height // 2)

    # 4. 이미지 자르기
    cropped_image = image[y_min:y_max, x_min:x_max]

    # 5. 자른 이미지를 1280x960으로 리사이즈
    resized_image = cv2.resize(cropped_image, target_size, interpolation=cv2.INTER_LINEAR)

    return resized_image

# YOLO 세그멘테이션 모델 추론 및 이미지 처리
def main():
    # YOLO 세그멘테이션 모델 로드
    model = YOLO("best.pt")

    # 입력 폴더 및 출력 폴더 설정
    input_folder = "magnet_imgs/normal/normal_jpeg"
    output_folder = "resized_images"
    os.makedirs(output_folder, exist_ok=True)

    # 입력 폴더의 모든 이미지 파일 처리
    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            # 입력 이미지 불러오기
            image_path = os.path.join(input_folder, file_name)
            image = cv2.imread(image_path)

            # YOLO 세그멘테이션 수행
            results = model(image)

            for result in results:
                # 1. 세그멘테이션 점 추출
                masks = result.masks.xy  # 세그멘테이션 결과 점들
                for mask in masks:
                    # mask는 (N, 2) 형태로 점 좌표들을 포함
                    points = mask.astype(int)

                    # 2. 꼭짓점 계산
                    top_left, bottom_left, top_right, bottom_right = calculate_corners_from_points(points)
                    print(f"Top-left: {top_left}, Bottom-left: {bottom_left}, Top-right: {top_right}, Bottom-right: {bottom_right}")

                    # 3. 네 꼭짓점을 포함하며 4:3 비율로 자르고 1280x960 크기로 맞추기
                    resized_image = crop_include_points_and_resize(
                        image,
                        points=[top_left, bottom_left, top_right, bottom_right],
                        target_size=(1280, 960)
                    )

                    # 4. 결과 이미지 저장
                    base_name, _ = os.path.splitext(file_name)
                    output_path = os.path.join(output_folder, f"resized_{base_name}.jpeg")
                    cv2.imwrite(output_path, resized_image)

if __name__ == "__main__":
    main()



0: 480x640 1 semi, 64.0ms
Speed: 3.5ms preprocess, 64.0ms inference, 213.4ms postprocess per image at shape (1, 3, 480, 640)
Top-left: (332, 284), Bottom-left: (332, 504), Top-right: (1252, 284), Bottom-right: (1250, 486)

0: 480x640 1 semi, 2.3ms
Speed: 0.8ms preprocess, 2.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
Top-left: (332, 284), Bottom-left: (332, 506), Top-right: (1254, 284), Bottom-right: (1250, 484)

0: 480x640 1 semi, 2.3ms
Speed: 0.7ms preprocess, 2.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
Top-left: (316, 284), Bottom-left: (330, 514), Top-right: (1244, 284), Bottom-right: (1258, 480)

0: 480x640 1 semi, 2.2ms
Speed: 0.6ms preprocess, 2.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
Top-left: (324, 284), Bottom-left: (324, 498), Top-right: (1250, 284), Bottom-right: (1250, 484)

0: 480x640 1 semi, 2.2ms
Speed: 0.7ms preprocess, 2.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640

In [11]:
from ultralytics import YOLO
import cv2
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt  

from preprocessing.single_image_enhance_tflite import zeroDCE



class MagnetProcessor:
    def __init__(self, model_path, input_folder, output_folder):
        self.model = YOLO(model_path)
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.results_list = []
        self.left_differences = []
        self.right_differences = []
        self.corner_coordinates = {}
        self.min_gaps = []
        self.outlier_images = set()  # 이상치로 검출된 이미지 목록을 저장할 set


        # 출력 폴더 생성
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    def enhance_contrast(self,image, method='clahe', params=None):
        """
        이미지의 대비를 강화하는 함수, ZeroDCE 포함
    
        Parameters:
        image: 입력 이미지 (파일 경로 또는 이미지 배열)
        method: 대비 강화 방법 ('clahe', 'histeq', 'gamma', 'zeroDCE' 중 선택)
        params: 각 방법에 대한 추가 매개변수 딕셔너리
    
        Returns:
        enhanced: 대비가 강화된 이미지
        """
        # 이미지가 경로로 주어진 경우 로드
        if isinstance(image, str):
            img = cv2.imread(image)
        else:
            img = image.copy()
    
        if params is None:
            params = {}
    
        if method == 'clahe':
            # CLAHE (Contrast Limited Adaptive Histogram Equalization) 적용
            clip_limit = params.get('clip_limit', 2.0)
            tile_grid_size = params.get('tile_grid_size', (8, 8))
    
            lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)
    
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
            cl = clahe.apply(l)
    
            enhanced = cv2.merge((cl, a, b))
            enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)
    
        elif method == 'histeq':
            # 일반 히스토그램 평활화
            lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)
    
            # L 채널에 대해 히스토그램 평활화 적용
            cl = cv2.equalizeHist(l)
    
            enhanced = cv2.merge((cl, a, b))
            enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)
    
        elif method == 'gamma':
            # 감마 보정
            gamma = params.get('gamma', 1.5)
            lookup_table = np.array([((i / 255.0) ** gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
            enhanced = cv2.LUT(img, lookup_table)
    
        elif method == 'zeroDCE':
            # ZeroDCE 적용
            if isinstance(image, str):
                enhanced = zeroDCE(img_path=image).numpy()
            else:
                temp_path = "temp_image_for_zerodce.jpeg"
                cv2.imwrite(temp_path, img)
                enhanced = zeroDCE(img_path=temp_path).numpy()
                os.remove(temp_path)
    
        else:
            # 지정되지 않은 경우 원본 이미지 반환
            enhanced = img
    
        return enhanced


    
    def find_boundary_points(self, mask, min_area=100, epsilon_factor=0.02):
        """
        마스크에서 4개의 코너 포인트를 찾고 이상치를 제거하는 함수
        """
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        largest_contour = max(contours, key=cv2.contourArea)
        epsilon = epsilon_factor * cv2.arcLength(largest_contour, True)
        approx = cv2.approxPolyDP(largest_contour, epsilon, True)
    
        if len(approx) > 4:
            rect = cv2.minAreaRect(largest_contour)
            box = cv2.boxPoints(rect)
            approx = np.intp(box)
        elif len(approx) < 4:
            x, y, w, h = cv2.boundingRect(largest_contour)
            approx = np.array([[x,y], [x+w,y], [x+w,y+h], [x,y+h]], dtype=np.float32)
    
        points = approx.reshape(-1, 2)
        ordered_points = self.order_points(points)
        return ordered_points

    def order_points(self, pts):
        """
        4개의 코너 포인트를 순서대로 정렬
        순서: [좌상, 우상, 우하, 좌하]
        """
        rect = np.zeros((4, 2), dtype="float32")
    
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]  # 좌상
        rect[2] = pts[np.argmax(s)]  # 우하
    
        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]  # 우상
        rect[3] = pts[np.argmax(diff)]  # 좌하
    
        return rect
 
    
    def draw_roi1_annotations(self, image, roi1_x, roi1_y, roi1_width, roi1_height, height, real_min_y, real_max_y):
        if height > 0:
            cv2.circle(image, (roi1_x + roi1_width//2, real_min_y), 3, (0, 255, 255), -1)
            cv2.circle(image, (roi1_x + roi1_width//2, real_max_y), 3, (0, 255, 255), -1)
            cv2.line(image, 
                    (roi1_x + roi1_width//2, real_min_y),
                    (roi1_x + roi1_width//2, real_max_y),
                    (0, 255, 255), 2)
            cv2.putText(image, f"Height: {height:.1f}px", 
                        (roi1_x + roi1_width + 10, roi1_y + roi1_height//2),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    def draw_roi2_annotations(self, image, roi2_result, roi2_x, roi2_y):
        if roi2_result is not None:
            gaps, top_points, bottom_points = roi2_result
            if gaps:
                min_gap = min(gaps)
                min_gap_idx = gaps.index(min_gap)
    
                min_gap_x = top_points[min_gap_idx][0]
                min_gap_top_y = top_points[min_gap_idx][1]
                min_gap_bottom_y = bottom_points[min_gap_idx][1]
    
                for (x, top_y), (_, bottom_y) in zip(top_points, bottom_points):
                    cv2.circle(image, (int(x), int(top_y)), 1, (255, 0, 0), -1)
                    cv2.circle(image, (int(x), int(bottom_y)), 1, (0, 255, 0), -1)
    
                cv2.line(image, 
                        (int(min_gap_x), int(min_gap_top_y)),
                        (int(min_gap_x), int(min_gap_bottom_y)),
                        (0, 255, 255), 2)
                cv2.putText(image, f"Min Gap: {min_gap:.1f}px",
                            (roi2_x + 10, roi2_y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    

    def process_roi1(self, image, roi1_x, roi1_y, roi1_width, roi1_height, current_image=None):  
        roi_img = image[roi1_y:roi1_y+roi1_height, roi1_x:roi1_x+roi1_width]
        gray_roi = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)
        bright_pixels = np.where(gray_roi >= 200)
    
        height = 0
        real_min_y = 0
        real_max_y = 0
    
        if len(bright_pixels[0]) > 0:
            clusters = self.cluster_pixels(bright_pixels[0])
            heights, min_max_y_pairs = self.calculate_heights(clusters, current_image)
    
            if heights:
                # 가장 큰 높이 선택
                max_height_idx = heights.index(max(heights))
                min_y, max_y = min_max_y_pairs[max_height_idx]
    
                real_min_y = min_y + roi1_y
                real_max_y = max_y + roi1_y
                height = real_max_y - real_min_y
    
        return height, real_min_y, real_max_y

    
    def process_roi2(self, image, roi2_x, roi2_y, roi2_width, roi2_height):
        try:
            roi2_img = image[roi2_y:roi2_y+roi2_height, roi2_x:roi2_x+roi2_width]
            
            if roi2_img.size == 0:  # ROI 영역이 비어있는지 확인
                return None

            #######################################3여기에 대비 강화 추가 함##############################
            # Enhance contrast using different methods and parameters
            #roi2_img = self.enhance_contrast(roi2_img, method='clahe', params={'clip_limit': 2.0, 'tile_grid_size': (8, 8)})
            #roi2_img = self.enhance_contrast(roi2_img, method='gamma', params={'gamma': 1.5})
            roi2_img = self.enhance_contrast(roi2_img, method='zeroDCE')
            # roi2_img = self.enhance_contrast(roi2_img, method='histeq')
    
            gray_roi2 = cv2.cvtColor(roi2_img, cv2.COLOR_BGR2GRAY)
            bright_pixels2 = np.where(gray_roi2 >= 200)
    
            if len(bright_pixels2[0]) > 0:
                return self.analyze_gaps(bright_pixels2, roi2_x, roi2_y)
            return None
    
        except Exception as e:
                print(f"Error in process_roi2: {str(e)}")
                return None

        except Exception as e:
            print(f"Error in process_roi2: {str(e)}")
            return None


    def cluster_pixels(self, y_coords):
        clusters = []
        current_cluster = [y_coords[0]]

        for y in y_coords[1:]:
            if y - current_cluster[-1] > 10:
                clusters.append(current_cluster)
                current_cluster = [y]
            else:
                current_cluster.append(y)
        clusters.append(current_cluster)
        return clusters
        
        
    def analyze_gaps(self, bright_pixels2, roi2_x, roi2_y):
        x_grouped_pixels = {}
        for x, y in zip(bright_pixels2[1], bright_pixels2[0]):
            if x not in x_grouped_pixels:
                x_grouped_pixels[x] = []
            x_grouped_pixels[x].append(y)

        gaps = []
        top_points = []
        bottom_points = []
        min_separation = 5

        for x in sorted(x_grouped_pixels.keys()):
            y_coords = sorted(x_grouped_pixels[x])
            clusters = []
            current_cluster = [y_coords[0]]

            for y in y_coords[1:]:
                if y - current_cluster[-1] > min_separation:
                    clusters.append(current_cluster)
                    current_cluster = [y]
                else:
                    current_cluster.append(y)
            clusters.append(current_cluster)

            if len(clusters) >= 2:
                top_max = max(clusters[0])
                bottom_min = min(clusters[-1])
                gap = bottom_min - top_max

                real_x = x + roi2_x
                real_top_y = top_max + roi2_y
                real_bottom_y = bottom_min + roi2_y

                gaps.append(gap)
                top_points.append((real_x, real_top_y))
                bottom_points.append((real_x, real_bottom_y))

        return (gaps, top_points, bottom_points) if gaps else None
        
    ''' 
    높이 측정
    '''
    def calculate_heights(self, clusters, current_image=None):
        heights = []
        min_max_y_pairs = []
        for cluster in clusters:
            min_y = min(cluster)
            max_y = max(cluster)
            cluster_height = max_y - min_y
            if cluster_height > 10:  # 기본적인 노이즈 필터링
                heights.append(cluster_height)
                min_max_y_pairs.append((min_y, max_y))

        return heights, min_max_y_pairs
    
    


    def draw_annotations(self, image, corners, roi1_data, roi2_data):
        # 코너 포인트 표시
        colors = [(0,0,255), (0,255,0), (255,0,0), (255,255,0)]
        for i, (x, y) in enumerate(corners.astype(int)):
            cv2.circle(image, (x, y), 4, colors[i], -1)

        # ROI 영역 및 측정값 표시
        if roi1_data:
            self.draw_roi1_annotations(image, *roi1_data)
        if roi2_data:
            self.draw_roi2_annotations(image, *roi2_data)

        return image


    

    def process_image(self, image_file):
        try:
            image_path = os.path.join(self.input_folder, image_file)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Failed to load image: {image_file}")
                return
    
            # 이미지의 상단 @@% 영역만 선택
            height = image.shape[0]
            bottom_limit = int(height * 0.70)  # 하단 15% 시작점

            # @@% 기준선 그리기 (빨간색 선)
            cv2.line(image, (0, bottom_limit), (image.shape[1], bottom_limit), (0, 0, 255), 2)
    
            # 상단 @@% 영역만 복사
            top_image = image[:bottom_limit, :].copy()
    
            # 상단 영역에서만 예측 수행
            results = self.model.predict(top_image, conf=0.3, iou=0.5, verbose=False)
    
            for result in results:
                if hasattr(result, 'masks') and result.masks is not None:
                    masks = result.masks.data.cpu().numpy()
    
                    for mask in masks:
                        mask = mask.astype(np.uint8)
                        mask = cv2.resize(mask, (image.shape[1], bottom_limit))
                        corners = self.find_boundary_points(mask)
    
                        # 코너 좌표 저장
                        self.corner_coordinates[image_file] = corners.astype(int)
    
                        # 4번 꼭지점(좌하단) 좌표 추출
                        bottom_left_point = corners[3].astype(int)
                        x4, y4 = bottom_left_point
    
                        # ROI1 좌표 계산
                        roi1_x = x4 - 30
                        roi1_width = 60
                        roi1_y = y4 - 300   # height 증가만큼 같이 증가시켜주면 됌
                        roi1_height = 300   # 숫자 올리면 밑으로 길어짐
    
                        # ROI2 좌표 계산
                        roi2_x = x4 + 240
                        roi2_width = 500
                        roi2_y = y4 - 290
                        roi2_height = 50

    
                        # ROI1 처리
                        height, real_min_y, real_max_y = self.process_roi1(
                            image, roi1_x, roi1_y, roi1_width, roi1_height, image_file
                        )
                        
                        # 실제 표시되는 높이 계산
                        display_height = real_max_y - real_min_y if height > 0 else 0
                        
                        # 높이 정보 저장 - 중복 저장 방지
                        if display_height > 0:  # 유효한 높이일 때만 저장
                            if image_file in self.corner_coordinates:  # 이미지가 이미 처리된 적이 있다면
                                idx = list(self.corner_coordinates.keys()).index(image_file)
                                if idx < len(self.left_differences):
                                    self.left_differences[idx] = display_height  # 기존 값 업데이트
                                    self.right_differences[idx] = display_height
                                else:
                                    self.left_differences.append(display_height)  # 새로운 값 추가
                                    self.right_differences.append(display_height)
                        
                        # 결과 저장
                        self.results_list.append(f"{image_file} - Height: {display_height:.2f} pixels")
                        
                        # ROI1 측정 결과 표시
                        if display_height > 0:
                            cv2.circle(image, (roi1_x + roi1_width//2, real_min_y), 3, (0, 255, 255), -1)
                            cv2.circle(image, (roi1_x + roi1_width//2, real_max_y), 3, (0, 255, 255), -1)
                            cv2.line(image, 
                                    (roi1_x + roi1_width//2, real_min_y),
                                    (roi1_x + roi1_width//2, real_max_y),
                                    (0, 255, 255), 2)
                            cv2.putText(image, f"Height: {display_height:.1f}px", 
                                      (roi1_x + roi1_width + 10, roi1_y + roi1_height//2),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)


                        
    
                        # ROI2 처리
                        roi2_result = self.process_roi2(
                            image, roi2_x, roi2_y, roi2_width, roi2_height
                        )
    
                        # ROI 영역 표시
                        cv2.rectangle(image, 
                                    (roi1_x, roi1_y), 
                                    (roi1_x + roi1_width, roi1_y + roi1_height), 
                                    (255, 255, 255), 2)
                        cv2.rectangle(image, 
                                    (roi2_x, roi2_y), 
                                    (roi2_x + roi2_width, roi2_y + roi2_height), 
                                    (0, 255, 0), 2)

    
    
                        # ROI2 결과 표시
                        if roi2_result is not None:
                            gaps, top_points, bottom_points = roi2_result
                            if gaps:
                                min_gap = min(gaps)
                                min_gap_idx = gaps.index(min_gap)
                                self.min_gaps.append(min_gap)
    
                                min_gap_x = top_points[min_gap_idx][0]
                                min_gap_top_y = top_points[min_gap_idx][1]
                                min_gap_bottom_y = bottom_points[min_gap_idx][1]
    
                                # Gap 포인트 표시
                                for (x, top_y), (_, bottom_y) in zip(top_points, bottom_points):
                                    cv2.circle(image, (int(x), int(top_y)), 1, (255, 0, 0), -1)
                                    cv2.circle(image, (int(x), int(bottom_y)), 1, (0, 255, 0), -1)
    
                                # 최소 gap 표시
                                cv2.line(image, 
                                        (int(min_gap_x), int(min_gap_top_y)),
                                        (int(min_gap_x), int(min_gap_bottom_y)),
                                        (0, 255, 255), 2)
                                cv2.putText(image, f"Min Gap: {min_gap:.1f}px",
                                          (roi2_x + 10, roi2_y - 10),
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
                        # 코너 포인트 표시
                        colors = [(0,0,255), (0,255,0), (255,0,0), (255,255,0)]
                        for i, (x, y) in enumerate(corners.astype(int)):
                            cv2.circle(image, (x, y), 4, colors[i], -1)
    
            # 결과 이미지 저장
            output_path = os.path.join(self.output_folder, f'ROI_{image_file}')
            cv2.imwrite(output_path, image)
            # print(f"Processed: {image_file}")
    
        except Exception as e:
            print(f"Error processing {image_file}: {str(e)}")
   

    def analyze_and_visualize_results(self):
        # 유효한 높이 값만 추출 (0이 아닌 값)
        valid_heights = [h for h in self.left_differences if h > 0]
    
        if not valid_heights:
            print("No valid height measurements found.")
            return
    
        # 기본 통계 계산
        mean_height = np.mean(valid_heights)
        std_height = np.std(valid_heights)
    
        print("\n=== Statistical Analysis ===")
        print(f"평균 Height: {mean_height:.2f} pixels")
        print(f"Standard Deviation: {std_height:.2f} pixels")
        print(f"최소 Height: {min(valid_heights):.2f} pixels")
        print(f"최대 Height: {max(valid_heights):.2f} pixels")
    
        # 이상치 탐지 (평균에서 2 표준편차 이상 벗어난 경우)
        threshold = 2.0  # 표준편차의 배수
        upper_bound = mean_height + (threshold * std_height)
        lower_bound = mean_height - (threshold * std_height)
    
        # 이상치 이미지 찾기
        outliers = []
        for image_name, height in zip(self.corner_coordinates.keys(), self.left_differences):
            if height > 0:  # 유효한 높이에 대해서만 검사
                if height > upper_bound or height < lower_bound:
                    outliers.append((image_name, height))
                    # ROI_ 접두사가 붙은 결과 이미지 이름을 저장
                    self.outlier_images.add(f'ROI_{image_name}')  # 이상치 이미지 저장
    
        # 이상치 결과 출력
        print("\n=== Outlier Analysis ===")
        print(f"Number of outliers detected: {len(outliers)}")
        if outliers:
            print("\nOutlier Images:")
            for image_name, height in outliers:
                deviation = abs(height - mean_height)
                std_deviations = deviation / std_height
                print(f"{image_name}: {height:.2f} pixels "
                      f"({std_deviations:.2f} standard deviations from mean)")
    
        # 히스토그램 시각화
        plt.figure(figsize=(10, 6))
        plt.hist(valid_heights, bins=20, edgecolor='black')
        plt.axvline(mean_height, color='r', linestyle='dashed', linewidth=2, label='Mean')
        plt.axvline(upper_bound, color='g', linestyle='dashed', linewidth=2, label='Upper Bound')
        plt.axvline(lower_bound, color='g', linestyle='dashed', linewidth=2, label='Lower Bound')
        plt.xlabel('Height (pixels)')
        plt.ylabel('Frequency')
        plt.title('Distribution of Height Measurements')
        plt.legend()
    
        # 히스토그램 저장
        plt.savefig(os.path.join(self.output_folder, 'height_distribution.png'))
        plt.close()
    
        # 이상치 이미지 별도 폴더로 복사
        outlier_folder = os.path.join(self.output_folder, 'outliers')
        if not os.path.exists(outlier_folder):
            os.makedirs(outlier_folder)

        for roi_image_name in self.outlier_images:
            # 처리된 ROI 이미지만 복사
            processed_src = os.path.join(self.output_folder, roi_image_name)
            processed_dst = os.path.join(outlier_folder, roi_image_name)
            if os.path.exists(processed_src):
                shutil.copy2(processed_src, processed_dst)
    
        for image_name in self.outlier_images:
            # 원본 이미지와 처리된 이미지 모두 복사
            src_path = os.path.join(self.input_folder, image_name)
            dst_path = os.path.join(outlier_folder, image_name)
            if os.path.exists(src_path):
                shutil.copy2(src_path, dst_path)
    
            # 처리된 이미지 복사
            processed_src = os.path.join(self.output_folder, f'ROI_{image_name}')
            processed_dst = os.path.join(outlier_folder, f'ROI_{image_name}')
            if os.path.exists(processed_src):
                shutil.copy2(processed_src, processed_dst)


    def process_all_images(self):
        # 처리할 이미지 파일 목록 생성
        image_files = [f for f in os.listdir(self.input_folder) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        total_images = len(image_files)
    
        # 각 이미지 처리
        for idx, image_file in enumerate(image_files, 1):
            # \r을 사용하여 커서를 줄의 시작으로 이동하고, end=''로 줄바꿈을 방지
            print(f"\rProcessing: {idx}/{total_images}", end='', flush=True)
            self.process_image(image_file)
    
        # 처리 완료 후 줄바꿈
        print("\nProcessing completed!")
    
        # 모든 이미지 처리가 완료된 후 결과 분석 실행
        self.analyze_and_visualize_results()

2025-01-09 11:43:06.363654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736422986.369842    6471 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736422986.371695    6471 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
# 사용 예시
if __name__ == "__main__":
    processor = MagnetProcessor(
        model_path='best_50.pt',
        input_folder='resized_images',
        output_folder='outputs/'
    )
    processor.process_all_images()

Processing: 10/1066

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1736422987.248450    6471 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Processing: 1066/1066
Processing completed!

=== Statistical Analysis ===
평균 Height: 272.41 pixels
Standard Deviation: 8.96 pixels
최소 Height: 144.00 pixels
최대 Height: 291.00 pixels

=== Outlier Analysis ===
Number of outliers detected: 11

Outlier Images:
resized_0473.jpeg: 211.00 pixels (6.85 standard deviations from mean)
resized_0136.jpeg: 247.00 pixels (2.84 standard deviations from mean)
resized_1021.jpeg: 249.00 pixels (2.61 standard deviations from mean)
resized_0481.jpeg: 173.00 pixels (11.09 standard deviations from mean)
resized_0734.jpeg: 144.00 pixels (14.33 standard deviations from mean)
resized_0276.jpeg: 291.00 pixels (2.07 standard deviations from mean)
resized_0480.jpeg: 203.00 pixels (7.74 standard deviations from mean)
resized_0732.jpeg: 150.00 pixels (13.66 standard deviations from mean)
resized_0733.jpeg: 165.00 pixels (11.98 standard deviations from mean)
resized_0137.jpeg: 228.00 pixels (4.96 standard deviations from mean)
resized_0476.jpeg: 239.00 pixels (3.73 s

In [6]:
!pwd


/workspace/semi_2
